Final Code with all feature engineering techniques 

BM + Lag + RWS + TBF + TLS + STL + Fourier

Cadiz dataset

In [1]:

import pandas as pd
import numpy as np
import time
import os
import psutil
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft

np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# 1. Load and Preprocess Data
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    df.columns = df.columns.str.replace('"', '').str.strip()
    if 'TIME' not in df.columns:
        raise KeyError("Column 'TIME' missing.")
    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        target_col = 'target'
    if target_col not in df.columns:
        raise KeyError("Target column not found.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col].astype(int)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)
    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()



    df.fillna(df.median(), inplace=True)
    return df, target_col

# 2. Feature Scaling
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y, scaler

# 3. MLP Model
def build_mlp_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    model.add(layers.Dense(num_units, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_units // 2, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# 4. Objective Function
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_mlp_model(
        input_shape=X_train.shape[1],
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# 5. Hyperparameter Optimization
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (30, 60),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# 6. Train and Evaluate
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_mlp_model(
        input_shape=X_train.shape[1],
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )
    start = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start

    start = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    testing_time = time.time() - start
    inference_time = testing_time / len(X_test)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, digits=4))
    print(confusion_matrix(y_test, y_pred))

    model.save("temp_model.h5")
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Testing Time (total): {testing_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# 7. Main
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cadiz_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cadiz_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Hyperparameters:", best_params)

    print("\n Training Final MLP Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9385    | 33.98     | 0.4803    | 51.96     | 89.47     |
| 2         | 0.9385    | 23.49     | 0.1624    | 31.74     | 115.2     |
| 3         | 0.9077    | 44.85     | 0.3832    | 30.62     | 125.1     |
| 4         | 0.9077    | 55.96     | 0.1849    | 35.45     | 49.61     |
| 5         | 0.9385    | 30.6      | 0.3099    | 42.96     | 59.96     |
| 6         | 0.9385    | 22.21     | 0.3256    | 31.47     | 115.9     |
| 7         | 0.9385    | 18.02     | 0.4178    | 37.39     | 96.29     |
| 8         | 0.9385    | 16.29     | 0.3761    | 56.62     | 69.8      |
| 9         | 0.9846    | 16.43     | 0.3942    | 59.5      | 113.4     |
| 10        | 1.0       | 19.73     | 0.1139    | 59.16     | 125.8     |
| 11        | 0.9692    | 33.07     | 0.2687    | 59.19     | 125.8     |
| 12  


===== Evaluation =====
Accuracy: 0.9440
              precision    recall  f1-score   support

           0     0.8000    0.5000    0.6154        16
           1     0.8816    0.9710    0.9241        69
           2     0.9926    0.9854    0.9890       137
           3     0.9000    0.9000    0.9000        10

    accuracy                         0.9440       232
   macro avg     0.8936    0.8391    0.8571       232
weighted avg     0.9423    0.9440    0.9401       232

[[  8   8   0   0]
 [  2  67   0   0]
 [  0   1 135   1]
 [  0   0   1   9]]

===== Model Metrics =====
Training Time: 15.96s
Testing Time (total): 0.38s
Inference/sample: 0.001623s
Model Size: 0.15 MB
Trainable Params: 10,314
RAM Usage: 734.82 MB


Cordoba dataset MLP

In [2]:

import pandas as pd
import numpy as np
import time
import os
import psutil
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft

np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# 1. Load and Preprocess Data
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    df.columns = df.columns.str.replace('"', '').str.strip()
    if 'TIME' not in df.columns:
        raise KeyError("Column 'TIME' missing.")
    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        target_col = 'target'
    if target_col not in df.columns:
        raise KeyError("Target column not found.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col].astype(int)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)
    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()



    df.fillna(df.median(), inplace=True)
    return df, target_col

# 2. Feature Scaling
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y, scaler

# 3. MLP Model
def build_mlp_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    model.add(layers.Dense(num_units, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_units // 2, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# 4. Objective Function
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_mlp_model(
        input_shape=X_train.shape[1],
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# 5. Hyperparameter Optimization
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (30, 60),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# 6. Train and Evaluate
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_mlp_model(
        input_shape=X_train.shape[1],
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )
    start = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start

    start = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    testing_time = time.time() - start
    inference_time = testing_time / len(X_test)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, digits=4))
    print(confusion_matrix(y_test, y_pred))

    model.save("temp_model.h5")
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Testing Time (total): {testing_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# 7. Main
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cordoba_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cordoba_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Hyperparameters:", best_params)

    print("\n Training Final MLP Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()




 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.8871    | 33.98     | 0.4803    | 51.96     | 89.47     |
| 2         | 0.9194    | 23.49     | 0.1624    | 31.74     | 115.2     |
| 3         | 0.8871    | 44.85     | 0.3832    | 30.62     | 125.1     |
| 4         | 0.8548    | 55.96     | 0.1849    | 35.45     | 49.61     |
| 5         | 0.8871    | 30.6      | 0.3099    | 42.96     | 59.96     |
| 6         | 0.9194    | 23.48     | 0.2078    | 33.0      | 115.1     |
| 7         | 0.9677    | 16.31     | 0.145     | 56.54     | 126.3     |
| 8         | 0.9839    | 29.18     | 0.2319    | 58.96     | 127.9     |
| 9         | 0.9355    | 62.74     | 0.2653    | 59.6      | 126.9     |
| 10        | 0.9355    | 35.19     | 0.4501    | 59.97     | 117.4     |
| 11        | 0.9194    | 16.18     | 0.1528    | 58.82     | 34.98     |
| 12  


===== Evaluation =====
Accuracy: 0.8736
              precision    recall  f1-score   support

           0     0.6667    0.1176    0.2000        17
           1     0.7049    0.9556    0.8113        45
           2     0.9897    0.9505    0.9697       101
           3     0.8462    1.0000    0.9167        11

    accuracy                         0.8736       174
   macro avg     0.8019    0.7559    0.7244       174
weighted avg     0.8754    0.8736    0.8502       174

[[ 2 15  0  0]
 [ 1 43  1  0]
 [ 0  3 96  2]
 [ 0  0  0 11]]

===== Model Metrics =====
Training Time: 9.37s
Testing Time (total): 0.26s
Inference/sample: 0.001497s
Model Size: 0.15 MB
Trainable Params: 10,606
RAM Usage: 954.82 MB


Jaen Dataset MLP

In [3]:

import pandas as pd
import numpy as np
import time
import os
import psutil
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft

np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ==============================
# 1. Data Loading and Preprocessing
# ==============================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'  # Change target column to 'fenologia_h1' for classification
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)

    # Subtract 1 from the target column to make it 0-indexed (0, 1, 2, 3)
    df[target_col] = df[target_col] - 1

    # Lag features
    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    # Rolling statistics
    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    # Time-based features for weekly data
    df['month'] = df.index.month                        # Useful for seasonal crop cycles
    df['weekofyear'] = df.index.isocalendar().week      # Week number in the year (1–52)
    df['year'] = df.index.isocalendar().year            # Optional: if data spans multiple years

    # Smooth cyclical encoding for week-of-year (annual periodicity)
    df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)

    # Additional Feature 1: Exponential Moving Average (EMA)
    df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()  # EMA with span 3
    df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()  # EMA with span 6

    # Additional Feature 2: Rolling Correlation
    # We compute the correlation between the target and 'month' as an example
    df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # Additional Feature 3: Fourier Transform (FFT) Features
    # Apply FFT to the target variable and extract real and imaginary components
    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))], axis=0)
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))], axis=0)

    # Fill missing values
    df.fillna(df.median(), inplace=True)

    return df, target_col

# 2. Feature Scaling
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y, scaler

# 3. MLP Model
def build_mlp_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    model.add(layers.Dense(num_units, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_units // 2, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# 4. Objective Function
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_mlp_model(
        input_shape=X_train.shape[1],
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# 5. Hyperparameter Optimization
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (30, 60),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# 6. Train and Evaluate
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_mlp_model(
        input_shape=X_train.shape[1],
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )
    start = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start

    start = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    testing_time = time.time() - start
    inference_time = testing_time / len(X_test)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, digits=4))
    print(confusion_matrix(y_test, y_pred))

    model.save("temp_model.h5")
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Testing Time (total): {testing_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# 7. Main
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Jaen_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Jaen_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Hyperparameters:", best_params)

    print("\n Training Final MLP Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.96      | 33.98     | 0.4803    | 51.96     | 89.47     |
| 2         | 0.96      | 23.49     | 0.1624    | 31.74     | 115.2     |
| 3         | 0.92      | 44.85     | 0.3832    | 30.62     | 125.1     |
| 4         | 0.86      | 55.96     | 0.1849    | 35.45     | 49.61     |
| 5         | 0.92      | 30.6      | 0.3099    | 42.96     | 59.96     |
| 6         | 1.0       | 16.71     | 0.4236    | 58.15     | 107.8     |
| 7         | 1.0       | 17.5      | 0.2856    | 59.64     | 125.8     |
| 8         | 1.0       | 18.61     | 0.1896    | 59.15     | 107.9     |
| 9         | 1.0       | 16.02     | 0.4024    | 59.33     | 93.7      |
| 10        | 0.98      | 17.07     | 0.4931    | 59.86     | 120.1     |
| 11        | 1.0       | 37.68     | 0.349     | 59.9      | 127.7     |
| 12  

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 



===== Evaluation =====
Accuracy: 0.9565
              precision    recall  f1-score   support

           0     1.0000    0.6154    0.7619        13
           1     0.9038    0.9400    0.9216        50
           2     0.9758    1.0000    0.9878       121

    accuracy                         0.9565       184
   macro avg     0.9599    0.8518    0.8904       184
weighted avg     0.9580    0.9565    0.9538       184

[[  8   5   0]
 [  0  47   3]
 [  0   0 121]]

===== Model Metrics =====
Training Time: 6.66s
Testing Time (total): 0.17s
Inference/sample: 0.000900s
Model Size: 0.13 MB
Trainable Params: 8,829
RAM Usage: 1113.25 MB


Sevilla MLP

In [4]:

import pandas as pd
import numpy as np
import time
import os
import psutil
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft

np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ==============================
# 1. Data Loading and Preprocessing
# ==============================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'  # Change target column to 'fenologia_h1' for classification
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)

    # Subtract 1 from the target column to make it 0-indexed (0, 1, 2, 3)
    df[target_col] = df[target_col] - 1

    # Lag features
    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    # Rolling statistics
    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    # Time-based features for weekly data
    df['month'] = df.index.month                        # Useful for seasonal crop cycles
    df['weekofyear'] = df.index.isocalendar().week      # Week number in the year (1–52)
    df['year'] = df.index.isocalendar().year            # Optional: if data spans multiple years

    # Smooth cyclical encoding for week-of-year (annual periodicity)
    df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)

    # Additional Feature 1: Exponential Moving Average (EMA)
    df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()  # EMA with span 3
    df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()  # EMA with span 6

    # Additional Feature 2: Rolling Correlation
    # We compute the correlation between the target and 'month' as an example
    df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # Additional Feature 3: Fourier Transform (FFT) Features
    # Apply FFT to the target variable and extract real and imaginary components
    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))], axis=0)
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))], axis=0)

    # Fill missing values
    df.fillna(df.median(), inplace=True)

    return df, target_col

# 2. Feature Scaling
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y, scaler

# 3. MLP Model
def build_mlp_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_shape,)))
    model.add(layers.Dense(num_units, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_units // 2, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# 4. Objective Function
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_mlp_model(
        input_shape=X_train.shape[1],
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# 5. Hyperparameter Optimization
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (30, 60),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# 6. Train and Evaluate
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_mlp_model(
        input_shape=X_train.shape[1],
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )
    start = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start

    start = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    testing_time = time.time() - start
    inference_time = testing_time / len(X_test)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, digits=4))
    print(confusion_matrix(y_test, y_pred))

    model.save("temp_model.h5")
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Testing Time (total): {testing_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# 7. Main
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\sevilla_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\sevilla_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Hyperparameters:", best_params)

    print("\n Training Final MLP Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()








 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 51.96     | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 31.74     | 115.2     |
| 3         | 0.9857    | 44.85     | 0.3832    | 30.62     | 125.1     |
| 4         | 0.9571    | 55.96     | 0.1849    | 35.45     | 49.61     |
| 5         | 0.9857    | 30.6      | 0.3099    | 42.96     | 59.96     |
| 6         | 1.0       | 16.71     | 0.4236    | 58.15     | 107.8     |
| 7         | 1.0       | 17.01     | 0.1588    | 30.56     | 88.97     |
| 8         | 0.9857    | 24.08     | 0.3354    | 32.56     | 116.3     |
| 9         | 0.9857    | 62.6      | 0.3838    | 59.82     | 67.9      |
| 10        | 0.9857    | 38.63     | 0.1642    | 54.06     | 41.26     |
| 11        | 0.9857    | 61.04     | 0.3825    | 43.5      | 127.9     |
| 12  

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step



===== Evaluation =====
Accuracy: 0.8744
              precision    recall  f1-score   support

           0     0.3750    0.3000    0.3333        10
           1     0.9054    0.8816    0.8933        76
           2     0.9516    0.9219    0.9365       128
           3     0.4118    0.7778    0.5385         9

    accuracy                         0.8744       223
   macro avg     0.6609    0.7203    0.6754       223
weighted avg     0.8882    0.8744    0.8787       223

[[  3   7   0   0]
 [  5  67   4   0]
 [  0   0 118  10]
 [  0   0   2   7]]

===== Model Metrics =====
Training Time: 6.05s
Testing Time (total): 0.24s
Inference/sample: 0.001055s
Model Size: 0.11 MB
Trainable Params: 6,543
RAM Usage: 1337.34 MB
